#Crawl

In [1]:
!pip install requests
!pip install bs4

In [2]:
import requests
from bs4 import BeautifulSoup

In [100]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

main_url = "https://tuyensinh.ptit.edu.vn/"
res = requests.get(main_url)
soup = BeautifulSoup(res.text, 'html.parser')

# Tập hợp tất cả các link bài viết
links = soup.find_all('a', href=True)
full_links = set([urljoin(main_url, a['href']) for a in links])

chunks = []

# Tập hợp lưu các nội dung đã xuất hiện
seen_contents = set()

def save_chunk(title, content, url):
    if not content.strip():
        return
    if "Tài liệu đào tạo" in title or "Đường dẫn liên kết" in title:
      return

    if "chuong-trinh-dao-tao" in url:
      content = f"[{title}] - {content}"

    # Cắt bỏ các dòng không cần thiết (chứa bản quyền, copyright,...)
    lines = content.strip().split('\n')

    # Loại bỏ các dòng không cần thiết
    clean_lines = []
    if "về chúng tôi" in title.lower():
        return
    for line in lines:
        lowered = line.lower()
        if any(keyword in lowered for keyword in [
            "copyright", "all rights reserved", "bản quyền",
            "hocviencongnghebuuchinhvienthong",
        ]):
            continue
        clean_lines.append(line)


    cleaned_content = "\n".join(clean_lines).strip()
    # print(clear_output)

    # Kiểm tra xem nội dung đã xuất hiện chưa
    content_signature = cleaned_content.lower().strip()  # chuẩn hóa
    if len(cleaned_content) > 10 and content_signature not in seen_contents and title:
        seen_contents.add(content_signature)
        chunks.append({
            "url": url,
            "title": title,
            "content": cleaned_content
        })


# full_links = ['https://daotao.ptit.edu.vn/chuong-trinh-dao-tao/nganh-cong-nghe-tai-chinh-fintech/']


for link in full_links:
    try:
        print(f"Crawling: {link}")
        res = requests.get(link, timeout=10)
        page_soup = BeautifulSoup(res.text, 'html.parser')

        # Bỏ header, footer, nav
        for tag in page_soup.select('header, footer, nav, .menu, .navbar, .footer, .header'):
            tag.decompose()

        content = page_soup.body or page_soup

        h1 = h2 = h3 = None
        buffer = []

        for tag in content.descendants:
            if tag.name in ['h1', 'h2', 'h3'] and tag.get_text(strip=True) != "Các học phần tự chọn":
                if buffer:
                    if not (h2 is None and h3 is None) or "hoc-bong" in link:
                      full_title = " - ".join(t for t in [h1, h2, h3] if t)
                      if(len(buffer) > 1 and "Học kỳ" in buffer[1]):
                        if len(buffer[1]) < 10:
                          buffer_1 = buffer[:2]  # Lấy 2 phần tử đầu

                          if len(buffer) > 2:
                              longest = max(buffer[2:], key=len)
                              buffer_1.append(longest)
                          buffer = buffer_1
                          full_title = full_title + " - " + buffer[1]

                      save_chunk(full_title, "\n".join(buffer), link)
                      buffer = []
                if tag.name == 'h1':
                    h1, h2, h3 = tag.get_text(strip=True), None, None
                elif tag.name == 'h2':
                    h2, h3 = tag.get_text(strip=True), None
                elif tag.name == 'h3':
                    h3 = tag.get_text(strip=True)


            elif tag.name in ['p', 'li', 'span', 'td', 'strong', 'br']:
                text = tag.get_text(strip=True)
                if text:
                    buffer.append(text)
            elif tag.name == 'a':
                href = tag.get('href')
                if href:
                    buffer.append(f"[{tag.get_text(strip=True)}]({href})")

            elif tag.name == 'table':
                rows = []
                for row in tag.find_all('tr'):
                    cells = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
                    if cells:
                        rows.append(" | ".join(cells))
                table_text = "\n".join(rows)
                if table_text:
                    buffer.append(table_text)

            elif tag.name == 'div' and (h1 or h2 or h3):
              if "chuong-trinh-dao-tao" in link or "hoc-bong" in link:
                  is_title_class = tag.get('class') and any(cls in tag['class'] for cls in ['tag', 'title', 'section-title', 'post-content'])
                  has_content_tags = tag.find(['p', 'span', 'li', 'td', 'strong', 'label'])

                  if is_title_class or has_content_tags:
                      for subtag in tag.descendants:
                          if isinstance(subtag, str):
                              continue  # Bỏ qua text node
                          new_chunk = False
                          if "học kỳ" in subtag.getText()[:8].lower():
                              new_chunk = True

                          # Nếu là h1/h2/h3 lồng bên trong
                          if (subtag.name in ['h1', 'h2', 'h3'] and subtag.get_text(strip=True) != "Các học phần tự chọn") or new_chunk:
                              if new_chunk and buffer:
                                  if(len(buffer) > 1 and "Học kỳ" in buffer[1]):
                                    if len(buffer[1]) < 10:
                                      full_title = " - ".join(t for t in [h1, h2, h3] if t) + " - " + buffer[1]
                                      buffer_1 = buffer[:2]  # Lấy 2 phần tử đầu
                                      if len(buffer) > 2:
                                          longest = max(buffer[2:], key=len)
                                          buffer_1.append(longest)
                                      buffer = buffer_1

                                    else:
                                      buffer = []
                                  else:
                                    full_title = " - ".join(t for t in [h1, h2, h3] if t)

                                  save_chunk(full_title, "\n".join(buffer), link)
                                  buffer = []
                                  text = subtag.get_text(separator="\n", strip=True)
                                  buffer.append(h1)
                                  buffer.append(text)
                                  new_chunk = False
                              elif "chuong-trinh-dao-tao" in link and buffer and not (h2 is None and h3 is None):
                                  full_title = " - ".join(t for t in [h1, h2, h3] if t)
                                  save_chunk(full_title, "\n".join(buffer), link)
                                  buffer = []
                              elif "hoc-bong" in link:
                                  full_title = " - ".join(t for t in [h1, h2, h3] if t)
                                  save_chunk(full_title, "\n".join(buffer), link)
                                  buffer = []
                              break  # Dừng xử lý div hiện tại
                      else:
                          # Nếu không có h1/h2/h3 bên trong
                          text = tag.get_text(separator="\n", strip=True)
                          if text:
                              buffer.append(text)
                  else:
                          # Nếu không có h1/h2/h3 bên trong
                          text = tag.get_text(separator="\n", strip=True)
                          if text:
                              buffer.append(text)


        if buffer:
            full_title = " - ".join(t for t in [h1, h2, h3] if t)
            save_chunk(full_title, "\n ".join(buffer), link)

    except Exception as e:
        print("❌ Error:", link, "\n", e)
        continue

Crawling: https://tuyensinh.ptit.edu.vn/2025/01/02/
Crawling: https://daotao.ptit.edu.vn/chuong-trinh-dao-tao/nganh-ky-thuat-dien-tu-vien-thong/
Crawling: https://daotao.ptit.edu.vn/chuong-trinh-dao-tao/chuong-trinh-logistics-va-quan-tri-chuoi-cung-ung/
Crawling: https://daotao.ptit.edu.vn/chuong-trinh-dao-tao/nganh-cong-nghe-thong-tin/
Crawling: https://tuyensinh.ptit.edu.vn/2025/02/27/
Crawling: https://maps.app.goo.gl/4ffWNferoqFBeBkN8
Crawling: https://daotao.ptit.edu.vn/chuong-trinh-dao-tao/nganh-khoa-hoc-may-tinh/
Crawling: https://daotao.ptit.edu.vn/chuong-trinh-dao-tao/nganh-ke-toan-chat-luong-cao-chuan-quoc-te-acca/
Crawling: https://khcn.ptit.edu.vn
Crawling: https://tuyensinh.ptit.edu.vn/thong-bao-ve-viec-dang-ky-nguyen-vong-xet-tuyen-dai-hoc-chinh-quy-tren-he-thong-ho-tro-tuyen-sinh-chung-cua-bo-giao-duc-va-dao-tao/
Crawling: https://tuyensinh.ptit.edu.vn/2025/06/03/
Crawling: https://tuyensinh.ptit.edu.vn/2025/05/30/
Crawling: https://tuyensinh.ptit.edu.vn/gioi-thieu/tong-

In [102]:
print(len(chunks))

903


In [106]:
data_chunked_file = "/content/drive/MyDrive/Colab Notebooks/Chatbot_TYP/data/data_tuyensinh5.json"

In [107]:
import json

with open(data_chunked_file, "w", encoding = 'utf-8') as f:
    json.dump(chunks, f,ensure_ascii=False, indent=2)